In [1]:
import akshare as ak
import pandas as pd
import datetime as dt
import numpy as np
import talib as tl
import matplotlib.pyplot as plt
import mplfinance as mpf

In [2]:
from pyecharts.charts import *
from pyecharts.components import Table
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts import options as opts
from pyecharts.globals import ThemeType

In [3]:
from util import *

In [4]:
# 参数设置
pd.set_option('display.float_format',lambda x:'%.2f' % x)

security = '601088'
end_day = '20200101'
period = 560

timeperiod=50
nbdevup=1
nbdevdn=1
matype=0
# 移动平均线最大天数
ma_max_day = 10

time_pattern='%Y-%m-%d'

pd.set_option('display.float_format',lambda x:'%.2f' % x)


In [5]:
end_day_dt= dt.datetime.strptime(end_day,'%Y%m%d')
price = get_price(security,end_day_dt,period)

In [6]:
# 获取Bolling带
price['upper'], price['middle'], price['lower'] = tl.BBANDS(
                price['close'].values,
                timeperiod=timeperiod,
                # number of non-biased standard deviations from the mean
                nbdevup=nbdevup,
                nbdevdn=nbdevdn,
                # Moving average type: simple moving average here
                matype=matype)

In [7]:
# 添加均线
append_ma(price)

In [8]:
index = 81
price.loc[index,'date']

'2018-10-19'

In [9]:
price.dropna(inplace=True)

hold = False
buy_index = None
hold_days = 0

money = start_money = 10000
volums = 0


for i in range(81,len(price)):
    index = price.iloc[i].name
    if (hold == False) & (price.loc[index-1,'upper'] < price.loc[index,'high']) & (price.loc[index-2,'upper'] > price.loc[index-1,'high']) & (price.loc[index-30,'low'] < price.loc[index,'low']):
        buy_price = price.loc[index-1,'upper']
        print('买入：',price.loc[index,'date'],price.loc[index-1,'upper'])
        hold = True
        buy_index = index
        volums = money/buy_price
        money = 0
        
    if hold:
        if (buy_index < index):
            if hold_days<ma_max_day:
                hold_days = hold_days + 1
            ma_price = price.loc[index-hold_days:index,'close'].mean()
            if price.loc[index,'low']<ma_price:
                money= volums * ma_price
                
                print('卖出：',price.loc[index,'date'], ma_price,money,money - start_money)

                hold = False
                buy_index = None
    

买入： 2019-06-17 16.066124142373134
卖出： 2019-06-18 16.155 10055.318791787786 55.31879178778581
买入： 2019-07-11 16.169788792168664
卖出： 2019-07-12 15.973333333333334 9933.151316852496 -66.84868314750383
买入： 2019-12-30 15.020622267754794


In [10]:
# K 线
y_data = price[['open','close','high','low']].values.tolist()
x_data = [i.replace('-','/') for i in price['date'].values]
kline = (Kline()
         .add_xaxis(x_data)
         .add_yaxis('K线', y_data)
         )


In [11]:
# bolling线
def get_line(x_data,y_data,name=''):
    return (Line().add_xaxis(x_data).add_yaxis(name,y_data,label_opts=opts.LabelOpts(is_show=False),is_symbol_show=False,is_smooth = True))


up_data = price['upper'].values
lower_data = price['lower'].values
middle_data = price['middle'].values

up_line = get_line(x_data,up_data,"上引线")
lower_line = get_line(x_data,lower_data,'下引线')
middle_line = get_line(x_data,middle_data,'中线')

In [12]:
kline.overlap(up_line)
kline.overlap(lower_line)
kline.overlap(middle_line)

In [13]:
for i in price.index:
    price.loc[i,'ma1'] = price.loc[i,['open','close','high','low']].mean()

In [14]:
# 均线
ma1_line = get_line(x_data,price['ma1'].values,'MA1')
kline.overlap(ma1_line)
ma5_line = get_line(x_data,price['ma5'].values,'MA5')
kline.overlap(ma5_line)
ma10_line = get_line(x_data,price['ma10'].values,'MA10')
kline.overlap(ma10_line)
ma20_line = get_line(x_data,price['ma20'].values,'MA20')
kline.overlap(ma20_line)
# kline.render_notebook()

In [15]:
# uu_data = price[(price['ma1']>price['ma5']) & (price['ma5']>price['ma20'])]
# for i in uu_data['date'].values:
#     print(i)

In [16]:

# ma1_line.overlap(ma5_line)
# ma1_line.overlap(ma20_line)
# ma1_line.render_notebook()

In [17]:
kline.set_global_opts(datazoom_opts=[  # 缩放选项
                                     opts.DataZoomOpts(
                                         is_show=False,
                                         type_="inside",
                                         xaxis_index=[0,1],
                                         # 初始的框选范围
                                         range_start=80,
                                         range_end=100,
                                     ),
                                     opts.DataZoomOpts(
                                         is_show=True,
                                         xaxis_index=[0,1],
                                         type_="slider",
                                         pos_top="95%",
                                         range_start=80,
                                         range_end=100,
                                     )
                                 ])

In [18]:
# 绘制成交量
bar = (Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
       .add_xaxis(x_data)
       .add_yaxis('成交量',price['turnover'].tolist(),label_opts=opts.LabelOpts(is_show=False))
       .set_global_opts(legend_opts=opts.LegendOpts(is_show=False))
      )
bar.render_notebook()

In [19]:
itheme="light"

grid_chart = Grid(
            init_opts=opts.InitOpts(
                width="900px", height="580px",
                animation_opts=opts.AnimationOpts(animation=True,animation_easing="linear"),
                theme=itheme, page_title="Pyecharts_Demo",
            )
        )
# 添加上图
grid_chart.add(
    kline,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="10%", height="60%"),
)
# 添加下图
grid_chart.add(
    bar,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="10%", pos_top="75%", height="16%"),
)

grid_chart.render_notebook()

In [20]:
price

,date,open,close,high,low,volume,turnover,amp_rate,quote_rate,quote_num,...,upper,middle,lower,ma5,ma10,ma20,ma30,ma60,ma120,ma1
119,2018-12-12,15.44,15.29,15.44,15.20,74313,142940043.00,1.59,1.06,0.16,...,16.40,15.77,15.15,15.08,15.04,15.19,15.53,15.63,14.92,15.34
120,2018-12-13,15.40,15.41,15.54,15.29,105036,203610795.00,1.64,0.78,0.12,...,16.37,15.75,15.13,15.22,15.13,15.17,15.51,15.65,14.92,15.41
121,2018-12-14,15.45,15.39,15.59,15.36,119969,233164113.00,1.49,-0.13,-0.02,...,16.35,15.73,15.12,15.27,15.18,15.15,15.48,15.67,14.91,15.45
122,2018-12-17,15.35,15.52,15.58,15.33,94488,183756852.00,1.62,0.84,0.13,...,16.34,15.72,15.11,15.35,15.22,15.12,15.46,15.68,14.91,15.44
123,2018-12-18,15.36,15.13,15.55,15.09,73891,141859111.00,2.96,-2.51,-0.39,...,16.30,15.69,15.08,15.35,15.21,15.10,15.42,15.70,14.90,15.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,2019-12-25,14.45,14.48,14.56,14.40,190940,334940048.00,1.10,-0.48,-0.07,...,15.06,14.61,14.16,14.56,14.45,14.32,14.30,14.80,15.38,14.47
372,2019-12-26,14.44,14.62,14.64,14.42,238220,419778864.00,1.52,0.97,0.14,...,15.03,14.59,14.15,14.55,14.50,14.35,14.31,14.77,15.36,14.53
373,2019-12-27,14.62,14.75,14.96,14.58,485836,866742736.00,2.60,0.89,0.13,...,15.02,14.59,14.15,14.56,14.56,14.38,14.34,14.76,15.35,14.73
374,2019-12-30,14.78,15.19,15.24,14.72,680494,1234217104.00,3.53,2.98,0.44,...,15.02,14.59,14.15,14.72,14.64,14.42,14.38,14.75,15.34,14.98


In [24]:
x_data

['2018/12/12',
 '2018/12/13',
 '2018/12/14',
 '2018/12/17',
 '2018/12/18',
 '2018/12/19',
 '2018/12/20',
 '2018/12/21',
 '2018/12/24',
 '2018/12/25',
 '2018/12/26',
 '2018/12/27',
 '2018/12/28',
 '2019/01/02',
 '2019/01/03',
 '2019/01/04',
 '2019/01/07',
 '2019/01/08',
 '2019/01/09',
 '2019/01/10',
 '2019/01/11',
 '2019/01/14',
 '2019/01/15',
 '2019/01/16',
 '2019/01/17',
 '2019/01/18',
 '2019/01/21',
 '2019/01/22',
 '2019/01/23',
 '2019/01/24',
 '2019/01/25',
 '2019/01/28',
 '2019/01/29',
 '2019/01/30',
 '2019/01/31',
 '2019/02/01',
 '2019/02/11',
 '2019/02/12',
 '2019/02/13',
 '2019/02/14',
 '2019/02/15',
 '2019/02/18',
 '2019/02/19',
 '2019/02/20',
 '2019/02/21',
 '2019/02/22',
 '2019/02/25',
 '2019/02/26',
 '2019/02/27',
 '2019/02/28',
 '2019/03/01',
 '2019/03/04',
 '2019/03/05',
 '2019/03/06',
 '2019/03/07',
 '2019/03/08',
 '2019/03/11',
 '2019/03/12',
 '2019/03/13',
 '2019/03/14',
 '2019/03/15',
 '2019/03/18',
 '2019/03/19',
 '2019/03/20',
 '2019/03/21',
 '2019/03/22',
 '2019/03/

In [21]:
df = price.copy()

In [60]:
# 图形分析
# 1. 在布林带下方
# 2. ma1上穿ma10 买入
# 3. ma1下穿ma10 卖出
df = price.copy()
df.dropna(inplace=True)

hold = False
buy_index = None
hold_days = 0

money = start_money = 10000
volums = 0
trade_times = 1
mark_data = []
own_moneys = [start_money,]
for index in df.index[1:]:
    # 在布林带下方，ma1上穿ma10
    if ((hold == False) 
        & (df.loc[index,'ma10'] < df.loc[index,'ma1']) 
        & (df.loc[index-1,'ma10'] > df.loc[index-1,'ma1'])
        & (df.loc[index,'ma1'] < df.loc[index,'lower'])):
        buy_df = df.loc[index,'ma1']
        print('买入：',df.loc[index,'date'],buy_df)
        hold = True
        buy_index = index
        volums = int(money/buy_df)
        money = 0
        
        
    if hold:
        if (buy_index < index):
            
            if df.loc[index,'ma1'] < df.loc[index,'ma10']:
                sell_df = df.loc[index,'ma1']
                money= round((volums * sell_df),2)
                own_moneys.append(money)
                profit = round((money - start_money),2)
                
                buy_day = df.loc[buy_index,'date'].replace('-','/')
                sell_day = df.loc[index,'date'].replace('-','/')

                print('卖出：',sell_day,sell_df,money,'盈利:',profit)
                mark_data.append(opts.MarkAreaItem(name="本次收益："+str(int(own_moneys[-1]-own_moneys[-2])), x=(buy_day, sell_day)))
                
                trade_times = trade_times + 1
                hold = False
                buy_index = None
                

买入： 2019-01-08 14.317499999999999
卖出： 2019/02/15 15.885000000000002 11087.73 盈利: 1087.73
买入： 2019-05-17 14.6625
卖出： 2019/05/20 14.424999999999999 10905.3 盈利: 905.3
买入： 2019-05-28 14.55
卖出： 2019/07/08 16.299999999999997 12208.7 盈利: 2208.7
买入： 2019-08-14 15.325
卖出： 2019/08/26 15.542499999999999 12371.83 盈利: 2371.83
买入： 2019-10-29 15.31
卖出： 2019/11/05 15.2225 12299.78 盈利: 2299.78
买入： 2019-11-25 14.5375
卖出： 2019/11/28 14.2375 12044.92 盈利: 2044.92
买入： 2019-12-09 14.255
卖出： 2019/12/12 14.124999999999998 11921.5 盈利: 1921.5


In [61]:
kline.set_series_opts(markarea_opts=opts.MarkAreaOpts(data=mark_data))

In [62]:
itheme="light"

grid_chart = Grid(
            init_opts=opts.InitOpts(
                width="900px", height="580px",
                animation_opts=opts.AnimationOpts(animation=True,animation_easing="linear"),
                theme=itheme, page_title="Pyecharts_Demo",
            )
        )
# 添加上图
grid_chart.add(
    kline,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="10%", height="60%"),
)
# 添加下图
grid_chart.add(
    bar,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="10%", pos_top="75%", height="16%"),
)

grid_chart.render_notebook()